# X-Ray Image Classification of Pneumonia in Pediatric Patients

<img src="images/title image.jpeg" style="width: 600px;">

## Overview

This project explores a dataset of x-ray images from pediatric patients with/without pneumonia. Pneumonia is a very common inflammatory condition that is found in the lungs, primarily in the air sacs when filled with fluid or pus. Symptoms can include cough, fever, chills, and difficulty breathing. Pneumonia can be life-threatening, but particularly to infants, children and people over the age of 65 ([Mayo Clinic](https://www.mayoclinic.org/diseases-conditions/pneumonia/symptoms-causes/syc-20354204)). 

The images in the dataset were selected from cohorts of patients from one to five years old from Guangzhou Women and Children's Medical Center. The data was provided by Kermany et al. on Mendeley through [Kaggle datasets](https://www.kaggle.com/paultimothymooney/chest-xray-pneumonia). All the chest x-ray images were screened for quality control, and then the diagnoses of the images were graded by two expert physicians before cleared for training. With the implementation of neural network models, it can help classify whether or not a given patient has pneumonia, given a chest x-ray image. 

## Business Understanding

The Children's Medical Center has asked for assistance in partially automating the diagnosis of pneumonia in their pediatric patients. Rather than finding the best possible accuracy on a model, a deep neural network that has been clearly iterated on can help our understanding of how these models and automation work in order to help doctors confidently and efficiently diagnosis pneumonia. Broadly speaking, this can also help our understanding of AI learning and its implementation in other parts of the medical field.

## Data Understanding

The data was organized into three folders: train, test, and val. Each folder contains sub-folders labeled as two categories, normal and pneumonia. Within the train set there are 5216 images between the two classes, 624 in test and only 16 images in val. Since the val set contained very few images, to better balance the validation set, I randomly selected images from the test folder and moved them to the respective class within val. The data_split notebook linked in this repository outlines this process. With the data augmentation, the redistribution of the test images into the validation set can be seen in the output as well.

Since the goal of this analysis is not to build the best model possible, but rather demonstrate an understanding of a working model. With the dataset being quite large, only training on a portion of the dataset will allow me to run models in a reasonable time first before training on the entire dataset.

In [1]:
#imports
import numpy as np
import pandas as pd
import random as rn
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import glob
import os

import tensorflow as tf
import tensorflow.random as tfr
import tensorflow.keras as keras
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras import layers
from tensorflow.keras.layers import Dense, Dropout, Flatten, Input
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array

import cv2

In [2]:
#creating path to the respective folder for each set in the data structure
data_path = os.path.join('chest-xray-pneumonia', 'chest_xray')
train_path = os.path.join(data_path, 'train')
test_path = os.path.join(data_path, 'test')
val_path = os.path.join(data_path, 'val')

In [3]:
#checking path
train_path

'chest-xray-pneumonia/chest_xray/train'

In [ ]:
#reading in the data images (https://www.kaggle.com/paramarthasengupta/pneumonia-diagnosis-eda-and-prediction?scriptVersionId=76497874&cellId=7)
labels = ['PNEUMONIA', 'NORMAL']
img_size = 150

def get_data(data_dir):
    data = []
    for label in labels:
        path = os.path.join(data_dir, label)
        class_num = labels.index(label)
        for img in os.listdir(path):
            try:
                img_arr = cv2.imread(os.path.join(path, img), cv2.IMREAD_GRAYSCALE)
                resized_arr = cv2.resize(img_arr, (img_size, img_size)) # Reshaping images to preferred size
                data.append([resized_arr, class_num])
            except Exception as e:
                print(e)
    return np.array(data)

In [ ]:
#getting the 3 datasets
train = get_data(train_path)
test = get_data(test_path)
val = get_data(val_path)

In [ ]:
#visualizing data distribution in train set, first creating a dataframe to help plot
train_df = pd.DataFrame(train, columns=['image', 'label'])

plt.figure(figsize=(18, 8))
sns.set_style("darkgrid")

sns.countplot(train_df['label'], palette = 'coolwarm')
plt.title('Train Data')

plt.show()

The training data appears imbalanced, with many more instances of pneumonia compared to normal. Data augmentation will help balance the distribution, this will be done in the data preparation section below.

In [ ]:
#sample image to see what the x-rays look like to begin with
plt.figure(figsize= (8, 8))
plt.imshow(train[0][0], cmap='gray')
plt.title(labels[train[0][1]]);

plt.figure(figsize= (8, 8))
plt.imshow(train[-1][0], cmap='gray')
plt.title(labels[train[-1][1]]);

## Data Preparation

The 3 datasets are split into train, test splits. Following this, image normalization and reshaping is done so that the images can be more easily interpreted once trained on a model. As previously mentioned, data augmentation is implemented.

In [ ]:
#splitting data into X_train, y_train, etc.
X_train = []
y_train = []

X_test = []
y_test = []

X_val = []
y_val = []

for feature, label in train:
    X_train.append(feature)
    y_train.append(label)

for feature, label in test:
    X_test.append(feature)
    y_test.append(label)
    
for feature, label in val:
    X_val.append(feature)
    y_val.append(label)

In [ ]:
#since these images are x-rays, they are a little more complex than simple images which is better suited for MLP
#cnn is probably better suited after a baseline model
#cnn takes 4-D tensors as input, which is what we have
#first, must normalize image data (since all pixel values will always be between 0 and 255)
X_train = np.array(X_train) / 255
X_val = np.array(X_val) / 255
X_test = np.array(X_test) / 255

In [ ]:
X_train = X_train.reshape(-1, img_size, img_size, 1)
y_train = np.array(y_train)

X_val = X_val.reshape(-1, img_size, img_size, 1)
y_val = np.array(y_val)

X_test = X_test.reshape(-1, img_size, img_size, 1)
y_test = np.array(y_test)

In [ ]:
#checking the shape of X_train array
X_train.shape

In [5]:
#data augmentation, applies random transformations on each image as it is passed to the model
#makes model more robust and saves on memory
#also prevents overfitting and handling imbalance in dataset
#instatiate ImageDataGenerator for data augmentation and then pass through each path to the 3 datasets
#https://studymachinelearning.com/keras-imagedatagenerator-with-flow_from_directory/

datagen = ImageDataGenerator(rescale=1/)

train_gen = datagen.flow_from_directory(train_path,
                                        target_size=(200, 200),
                                        color_mode='grayscale',
                                        shuffle=True,
                                        seed=42,
                                        class_mode='binary',
                                        batch_size=32)
test_gen = datagen.flow_from_directory(test_path,
                                       target_size=(200, 200),
                                       color_mode='grayscale',
                                       shuffle=False,
                                       seed=42,
                                       class_mode=None,
                                       batch_size=1)
val_gen = datagen.flow_from_directory(val_path,
                                      target_size=(200, 200),
                                      color_mode='grayscale',
                                      shuffle=True,
                                      seed=42,
                                      class_mode='binary',
                                      batch_size=32)

Found 5216 images belonging to 2 classes.
Found 511 images belonging to 2 classes.
Found 129 images belonging to 2 classes.


In [ ]:
# train_datagen = ImageDataGenerator(rescale=1. / 255,
#                                    samplewise_center=True,
#                                    samplewise_std_normalization=True,
#                                    rotation_range=15,
#                                    width_shift_range=0.2,
#                                    height_shift_range=0.2,
#                                    shear_range=0.2,
#                                    zoom_range=0.2,
#                                    horizontal_flip=True)
# test_datagen = ImageDataGenerator(rescale=1. / 255,
#                                   featurewise_center=True,
#                                   featurewise_std_normalization=True)

# train_generator = train_datagen.flow_from_directory(train_path,
#                                                     target_size=(150, 150),
#                                                     batch_size=32,
#                                                     class_mode='binary')
# test_generator = test_datagen.flow_from_directory(test_path,
#                                                   target_size=(150, 150),
#                                                   batch_size=32,
#                                                   class_mode='binary')
# val_generator = test_datagen.flow_from_directory(val_path,
#                                                  target_size=(150, 150),
#                                                  batch_size=32,
#                                                  class_mode='binary')

In [ ]:
# #displaying random images to see how they look after creating the generators
# print('Display Random Images')

# # Adjust the size of your images
# plt.figure(figsize=(20,10))

# for i in range(12):
#     num = rn.randint(1,30)
#     plt.subplot(3,4, i + 1)
    
#     x,y = train_generator.__getitem__(num)
    
#     plt.imshow(x[num],cmap='gray')
#     plt.axis('off')
    
# # Adjust subplot parameters to give specified padding
# plt.tight_layout()

In [ ]:
# #visualize examples from the train dataset #need to add label
# plt.figure(figsize=(5, 5))
# plt.imshow(train[0][0])

In [ ]:
# plt.imshow(X_train[0], cmap=plt.get_cmap('gray'))
# plt.show()

In [ ]:
# #checking the shape/format of our input
# X_train

In [ ]:
# #since this is currently a list, change to arrays
# X_train = np.array(X_train)
# y_train = np.array(y_train)

# X_test = np.array(X_test)
# y_train = np.array(y_train)

# X_val = np.array(X_val)
# y_val = np.array(y_val)

In [ ]:
# print('Train Shape:', X_train.shape)
# print('Test Shape:', X_test.shape)

In [ ]:
# datagen = ImageDataGenerator(featurewise_center=False,
#                              featurewise_std_normalization=False,
#                              rotation_range=20,
#                              width_shift_range=0.2,
#                              height_shift_range=0.2,
#                              horizontal_flip=True)
# datagen.fit(X_train)

## Modeling

### Baseline Model

In [14]:
#baseline model (helpful code from Joél study group)
def baseline_model():
    layers = [
        Input(shape=(200,200,1)),
        Flatten(),
        Dense(100, activation='relu'),
        Dense(1, activation='sigmoid'),
    ]
    model = Sequential(layers)
    model.compile(loss='binary_crossentropy',
                  optimizer='adam', #gradient descent, moving average
                  metrics=['accuracy'])
    return model

In [15]:
#initialize the model and pass in 1 of the train images to set the shape of input images
model_1 = baseline_model()

In [16]:
model_1.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 40000)             0         
_________________________________________________________________
dense (Dense)                (None, 100)               4000100   
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 101       
Total params: 4,000,201
Trainable params: 4,000,201
Non-trainable params: 0
_________________________________________________________________


In [19]:
step_size_train = train_gen.n // train_gen.batch_size
step_size_val = val_gen.n // val_gen.batch_size
model_1.fit_generator(train_gen,
                      validation_data=val_gen,
                      epochs=10,
                      steps_per_epoch=step_size_train,
                      validation_steps=step_size_val)

/opt/anaconda3/envs/learn-env/lib/python3.8/site-packages/keras/engine/training.py:1972: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/10
163/163 [==============================] - 36s 218ms/step - loss: 138.5949 - accuracy: 0.7140 - val_loss: 0.6822 - val_accuracy: 0.6172
Epoch 2/10
163/163 [==============================] - 37s 225ms/step - loss: 0.6551 - accuracy: 0.7429 - val_loss: 0.6723 - val_accuracy: 0.6172
Epoch 3/10
163/163 [==============================] - 40s 245ms/step - loss: 0.6315 - accuracy: 0.7429 - val_loss: 0.6643 - val_accuracy: 0.6250
Epoch 4/10
163/163 [==============================] - 42s 258ms/step - loss: 0.6136 - accuracy: 0.7429 - val_loss: 0.6618 - val_accuracy: 0.6250
Epoch 5/10
163/163 [==============================] - 38s 235ms/step - loss: 0.6002 - accuracy: 0.7429 - val_loss: 0.6620 - val_accuracy: 0.6250
Epoch 6/10
163/163 [==============================] - 40s 243ms/step - loss: 0.5906 - accuracy: 0.7429 - val_loss: 0.6691 - val_accuracy: 0.6172
Epoch 7/10
163/163 [==============================] - 38s 232ms/step - loss: 0.5838 - accuracy: 0.7429 - val_loss: 0.6671 - val_

In [22]:
#evaluating model performance
score = model_1.evaluate_generator(test_gen)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 0.0
Test accuracy: 0.0


In [23]:
score = model_1.evaluate_generator(val_gen)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 0.6825205087661743
Test accuracy: 0.6201550364494324


In [ ]:
# model_1 = Sequential()
# model_1.add(Dense(64, activation='relu', input_shape=(150, 150, 1)))
# model_1.add(Dense(10, activation='sigmoid'))

In [ ]:
# model_1.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
# model_1.fit_generator(train_it, steps_per_epoch=16, validation_data=val_it, validation_steps=8)

In [ ]:
# model_1.summary()

In [ ]:
# model_1.fit(train_generator, validation_data=test_generator)

In [ ]:
# #fit the model
# base = model_1.fit(datagen.flow(X_train, y_train, batch_size=32),
#                    epochs=10,
#                    validation_)

### Basic Convolutional Neural Network (CNN) Model

In [24]:
def conv_1():
    layers = [
        Input(shape=(200,200,1)),
        Conv2D(32, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        Flatten(),
        Dense(128, activation='relu'),
        Dense(1, activation='sigmoid'),
    ]
    model = Sequential(layers)
    model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])
    return model

In [25]:
model_2 = conv_1()

In [26]:
model_2.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 198, 198, 32)      320       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 99, 99, 32)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 313632)            0         
_________________________________________________________________
dense_2 (Dense)              (None, 128)               40145024  
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 129       
Total params: 40,145,473
Trainable params: 40,145,473
Non-trainable params: 0
_________________________________________________________________


In [27]:
model_2.fit_generator(train_gen,
                      validation_data=val_gen,
                      epochs=10,
                      steps_per_epoch=step_size_train,
                      validation_steps=step_size_val)

Epoch 1/10
163/163 [==============================] - 76s 465ms/step - loss: 144.2521 - accuracy: 0.8614 - val_loss: 15.9288 - val_accuracy: 0.7188
Epoch 2/10
163/163 [==============================] - 94s 575ms/step - loss: 1.0335 - accuracy: 0.9559 - val_loss: 6.3443 - val_accuracy: 0.8438
Epoch 3/10
163/163 [==============================] - 87s 530ms/step - loss: 0.3967 - accuracy: 0.9753 - val_loss: 9.8293 - val_accuracy: 0.7969
Epoch 4/10
163/163 [==============================] - 89s 543ms/step - loss: 0.2645 - accuracy: 0.9822 - val_loss: 14.4991 - val_accuracy: 0.7266
Epoch 5/10
163/163 [==============================] - 89s 546ms/step - loss: 0.2347 - accuracy: 0.9837 - val_loss: 21.1264 - val_accuracy: 0.7031
Epoch 6/10
163/163 [==============================] - 98s 603ms/step - loss: 0.1600 - accuracy: 0.9883 - val_loss: 18.0933 - val_accuracy: 0.7188
Epoch 7/10
163/163 [==============================] - 104s 638ms/step - loss: 0.0606 - accuracy: 0.9950 - val_loss: 21.6365 

In [30]:
score = model_2.evaluate_generator(test_gen)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 0.0
Test accuracy: 0.0


In [31]:
score = model_2.evaluate_generator(val_gen)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 10.3269681930542
Test accuracy: 0.7674418687820435


### Deeper CNN Model

In [ ]:
def conv_2():
    layers = [
        Input(shape=(200,200,1)),
        Conv2D(32, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        Flatten(),
        Dense(128, activation='relu'),
        Dense(1, activation='sigmoid'),
    ]
    model = Sequential(layers)
    model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])
    return model

In [ ]:
model_3 = conv_2()

### Transfer Learning